Import Classes

In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import Preprocessing
import imgaug.augmenters as iaa
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras import layers, models
from keras import applications
import matplotlib.pyplot as plt

Defining Random Image Augmentation

In [ ]:
random_augmenter = iaa.Sequential([
    # Horizontal Flip
    iaa.Fliplr(0.5),

    # Vertical Flip
    iaa.Flipud(0.5),

    # Multiply
    iaa.Multiply((0.8, 1.2)),

    # Linear Contrast
    iaa.LinearContrast((0.6, 1.4)),

    # Affine
    iaa.Affine(translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
               rotate=(-30,30),
               scale=(0.5, 1.5)),

    
])

Split Images and Labels into two different arrays with Augmentation (imgaug) and Pre-processing (GrabCut + CLAHE)

In [ ]:
path = 'Dataset/'
dataset_path = os.listdir('Dataset')
image_size = 224
images = []
labels = []

for label in dataset_path:
    data_path = path + str(label)
    filenames = [i for i in os.listdir(data_path)]

    for filename in filenames:
        img = cv2.imread(data_path + '/' + filename)

        # Preprocessing the image
        img = Preprocessing.preprocess(img)
        img = cv2.resize(img, (image_size, image_size))
        
        # image augmentation (horizontal and vertical flips)
        images.append(img)
        labels.append(label)
        cv2.imshow("Original", img)

        img = horizontal_flip(image=img)
        images.append(img)
        labels.append(label)
        cv2.imshow("Horizontal Flip", img)

        img = vertical_flip(image=img)
        images.append(img)
        labels.append(label)
        cv2.imshow("Horizontal & Vertical Flip", img)

        img = horizontal_flip(image=img)
        images.append(img)
        labels.append(label)
        cv2.imshow("Vertical Flip", img)
        cv2.waitKey(0)

images = np.array(images)
images = images.astype('float32')

Creating Dataframe for class labels

In [ ]:
dataframe = pd.DataFrame(data=labels, columns=['Labels', 'Image'])

Encoding the Labels + Splitting the dataframe to train-test sets (80-20)

In [ ]:
df_labels = dataframe['Labels'].values
print(df_labels)

df_labelEncoder = LabelEncoder()
df_labels = df_labelEncoder.fit_transform(df_labels)
print(df_labels)

df_labels = df_labels.reshape(-1, 1)
ct = ColumnTransformer([('my_ohe', OneHotEncoder(), [0])], remainder='passthrough')
Y = ct.fit_transform(df_labels)

images, Y = shuffle(images, Y, random_state=0)
train_x, test_x, train_y, test_y = train_test_split(images, Y, test_size=0.2, random_state=0)

Create the EfficientNetB0 model

In [ ]:
NUM_CLASSES = len(dataset_path)
IMG_SIZE = image_size
size = (IMG_SIZE, IMG_SIZE)

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

outputs = applications.EfficientNetB0(include_top=True, weights=None, classes=NUM_CLASSES)(inputs)

model = models.Model(inputs, outputs)

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

model.summary()

Train model by 30 epochs

In [ ]:
hist = model.fit(train_x, train_y, epochs=30, verbose=2)

Predicting an image from trained model

In [ ]:
pred_img_path = "Test1.JPG" # Input Image

pred_img = cv2.imread(pred_img_path)
pred_img = Preprocessing.preprocess(pred_img)
pred_img = cv2.resize(pred_img, (224, 224))

test_img = np.expand_dims(pred_img, axis=0)

prediction = model.predict(test_img) 
prediction

Exporting model

In [ ]:
import joblib
joblib.dump(model, 'Model_SegmClahe')

Importing model + Predicting an image / Obtaining test accuracy

In [ ]:
import joblib
import cv2
import Preprocessing
import numpy as np

file_model = joblib.load('Full')
# pred_img_path = "OnlineHealthy.JPG" # Input Image

# pred_img = cv2.imread(pred_img_path)
# pred_img = Preprocessing.preprocess(pred_img)
# pred_img = cv2.resize(pred_img, (224, 224))

# test_img = np.expand_dims(pred_img, axis=0)

# prediction = file_model.predict(test_img) 
# prediction


preds = file_model.evaluate(test_x, test_y)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

Testing Area

In [ ]:
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.title("Model Accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["Train", "validation"], loc="upper left")
    plt.show()

plot_hist(hist)

preds = model.evaluate(test_x, test_y)
print("Loss = " + str(preds[0]))
print("Test Accuracy = " + str(preds[1]))

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# predict probabilities for test set
yhat_classes = file_model.predict(test_x, verbose=0)
# predict crisp classes for test set
# yhat_classes = np.argmax(file_model.predict(test_x), axis=-1)
# reduce to 1d array
# yhat_probs = yhat_probs[:, 0]
yhat_classes = yhat_classes[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_y, yhat_classes)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_y, yhat_classes)
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_y, yhat_classes)
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test_y, yhat_classes)
print('F1 score: %f' % f1)